<h1 style="text-align: center;">Pytorch Basics: Tensor and Autograd</h1>

## 1. Introduction  
This notebook is a part of a part of my journey to learn Pytorch and deep learning. It serves as a reference for me and others who are interested in learning Pytorch. In this notebook, I will cover two important concepts in Pytorch: Tensor and Autograd that are, to my experience, the core of Pytorch. Along with the concepts, I will also provide some examples to illustrate how to use them.
## 2. Tensor
### 2.1 What is Tensor?
In machine learning, a tensor is an object that store collection of numbers that represent features of intersted objects or events. For example: a tensor can store iformation about height, weight, and age of a person. Each attribute is a dimention of the tensor. If a dimension of a tensor has one element, it is called a scalar. If a dimension of a tensor has more than one element, it is called a vector. If a tensor have more than one dimensions, it is called a multi-dimensional tensor. Two dimensions tensor is called a matrix.
### 2.2 How to create a tensor


In [13]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
plt.style.use( 'seaborn-v0_8-whitegrid') # set the style for the plot, this optional
from torchviz import make_dot # for visualizing the computational graph
from IPython.display import Image # for displaying


# initialise tensor from data
data = [[1, 2], [3, 4]]
tensor_data = torch.tensor(data)
print(f"this is the tensor from data:\n {tensor_data}")

# from numpy type
np_array = np.array([[1, 2], [3, 4]])
tensor_np = torch.from_numpy(np_array)
print(f"this is the tensor from numpy:\n {tensor_np}")


In [7]:
id(np_array) == id(tensor_np.numpy())  # True

In [13]:
a = np.array(1)
b = torch.from_numpy(a)
print(np.shares_memory(a, b.numpy()))  # Check if a and b share memory

In [ ]:
# id(a) == id(b) returns False because they are different objects in memory
# but they share the same underlying data.
print(id(a) == id(b))  # False

## 3. Fundamental of Autograd

### 3.1 What is Gradient Descent?
Before diving into Autograd, let's talk about machine learning training process. In context of machine learning, an initial model will take inputs and produce outputs. Then, the outputs are compared with the ground truth to see how well the model performs. The difference between the model's outputs and ground truth is called loss. Depending on nature of the outputs, different methods will be use to calculate the loss. For example: if the output is continuous, Mean Square Error or Mean Absolute Error can be used. If the output is categorical, cross entropy can be use *(I reserve topic about the loss in seperate Notebook)*.

The goal of machine learning is to minimise the loss. To achive this purpose, the model should change/update their weights (Coefficients in traditional modeling language) in way that will decrease the loss. To guide the change, the model should know direction of change. The direction of change is determined by the gradient of the loss function. The gradient of the loss function is calculated by taking the derivative of the loss function with respect to the weights. The gradient is a vector that points to the direction of the steepest increase of the loss function. To decrease the loss, the model should move in the opposite direction of the gradient. It is called **Gradient Descent**. Now, we know the direction to change the weights, but we also need to know how much to change the weights. This quantity is called learning rate. The update rule is as follows:

$$
\begin{align}
\theta_{new} = \theta_{old} - \alpha \nabla L
\end{align}
$$
- $\theta_{new}$: new weights after the update
- $\theta_{old}$: current weights before the update
- $\alpha$: learning rate, a small scalar value that controls the step size of the update
- $\nabla L$: gradient of the loss function with respect to the weights, indicating the direction and rate of the steepest increase in loss


## 3.2 What is Autograd?
Mathematically, the gradient of a function is the vector of partial derivatives of the function with respect to its parameters. There are two words that often confuse people *(like me)*: gradient and derivative. In short, gradient is a vector of partial derivatives. It is used for functions of multiple varibales. Derivative is a scalar value that indicates that rate of change of a function with respect to one variable. Following is examples to illustrate the difference between gradient and derivative:
- Derivative: $\frac{d}{dx} f(x) = f'(x)$
    - For function $f(x) = x^2$, the derivative is $f'(x) = 2x$; if x = 2, then $f'(2) = 4$
    - The value of $f'(2) = $ is a scalar value and is translated as the rate of change of the function $f(x) = x^2$ at x = 2 is 4. It means increase in x leads to increase in f(x) by 4 times. Then if we want do decrease f(x), we should decrease x.
- Gradient: if we have function $f(x_{1}, x_{2}, x_{3})$
    - Gradient is writen as: $\nabla f = (\frac{\nabla{f}}{\nabla{x_{1}}}, \frac{\nabla{f}}{\nabla{x_{2}}}, \frac{\nabla{f}}{\nabla{x_{3}}})$.
    - For example: if $f(x_{1}, x_{2}, x_{3}) = x_{1}^2 + x_{2}^2 + x_{3}^2$, then $\nabla f = (2x_{1}, 2x_{2}, 2x_{3})$ If $x_1 = 1; x_2 = 2; x_3 = 3$, then $\nabla f = (2, 4, 6)$. the $f' = 2^2 + 4^2 + 6^2 = 56$.
    - The gradient result can be interpreted as if we increase $x_1$ by 1, $x_2$ by 2, and $x_3$ by 3, the function $f(x_{1}, x_{2}, x_{3})$ will increase by 56.

The mathematical details of gradient and derivative are not focus of this notebook. Practically, we need to know that gradient points in the direction where the function increase fastest. For multiple variables function it is called gradient. For single varibale function, it is called derivative. The scope of this notebook is to show how to calculate gradient and update parameter using Autograd in Pytorch.

### 3.3 How to use Autograd in Pytorch
Let's start with a simple function $y = \sin(x) $. So we start with creating a tenssor x and calculate y. Then we calculate the gradient of y with respect to x. Codes implementation is presented in following code cell.

In [8]:
# create a tensor with random values
x_tensor = torch.linspace(0, 10, 100, requires_grad=True)
y_tensor = torch.sin(x_tensor)

# convert tensor to numpy
fig, ax = plt.subplots()
ax.plot(x_tensor.detach(), y_tensor.detach(), label='sin(x)')
ax.legend()
plt.show()

There is two special notes to be aware of:
- When I create x_tensor, I set requires_grad = True. This option tells Pytorch to track the gradient of x_tensor and any computation that involves x_tensor. The x_tensor is called leaf tensor. The gradient of leaf tensor is calculated by calling backward() method on output tensor. The gradient is stored in the grad attribute of the leaf tensor. No gradient is stored in the intermediate tensor. Let's explore x_tensor 

In [9]:
# explorer x_tensor
print(f"This is x_tensor: {x_tensor}")
print(f'along with each element of x_tensor ther is an requires_grad attribute=True: indicate that this tensor will be used to compute gradient')
print(f'the x_tensor has shape: {x_tensor.shape} and data type: {x_tensor.dtype} and stored in device: {x_tensor.device}')
print(f"This is grad_fn, a backward function of how to get: {x_tensor.grad_fn}")
print(f"This is accumulation grad associate with leaf tensor x_tensor: {x_tensor.grad}")

In [10]:
# do same with y_tensor
print(f"This is y_tensor: {y_tensor}")
print(f'along with each element of y_tensor ther is an requires_grad attribute=True: indicate that this tensor will be used to compute gradient')
print(f'the y_tensor has shape: {y_tensor.shape} and data type: {y_tensor.dtype} and stored in device: {y_tensor.device}')
print(f"This is grad_fn, a backward function of how to get: {y_tensor.grad_fn}")
print(f"This is accumulation grad associate with y_tensor: {y_tensor.grad}")

For the y_tensor, we have a slightly different stituation. First, there is no required_grad = True. This means that Pytorch does not track gradient of y_tensor. If we try to access gradient of y_tensor, it returns None and raise usewarning that gradient is nto available for non-leaf tensor. if we want to calculate gradient of y_tensor uses retain_grad() method on non-leaf tensor.

Secondly, y_tensor have grad_fn = <SinBackward0 object at 0x7f1badfac790> associate with it. It saying that the y_tensor is created by using sin function.
Let's visualise this concept in the following code cell.

In [ ]:
make_dot(y_tensor, params=dict(x=x_tensor)).render("plots/sinx_tensor", format="png")
Image("plots/sinx_tensor.png")

The graph telling to us that y_tensor is created by applying sin function on x_tensor. The x_tensor gradient is accumulated; both x_tensor and y_tensor have same shape of 100  *(we create a linear space with 100 values taking from 0 to 10)*.

Let's make the current computation a little bit more complex by adding additional computation.

In [15]:
z = 2*y_tensor + 1
g = z.sum()
make_dot(g, params=dict(x=x_tensor)).render("sum_z", format="png")
Image("sum_z.png")

In [16]:
# calculate the gradient and check the grad attribute of x_tensor
g.backward()
print(x_tensor.grad)

In [17]:
fig, ax = plt.subplots()
ax.plot(x_tensor.detach(), y_tensor.detach(), label='sin(x)')
ax.plot(x_tensor.detach(), z.detach(), label='2^y_tensor)+1')
ax.plot(x_tensor.detach(), x_tensor.grad.detach(), label='gradient')
ax.legend()
plt.show()